### Twilight Struggle

In [1]:
from twilight_ui import *

### Game Instance

In [ ]:
u = UI()
u.run()

Initalising game..
> new
Starting new game.
----- USSR Player: -----
Place starting influence.
Remaining influence: 1
Available options:
    7 Finland
   14 Austria
   16 East_Germany
   17 Poland
   18 Czechoslovakia
   19 Hungary
   20 Yugoslavia
   21 Romania
   22 Bulgaria
> m 14
Selected: Austria
----- USSR Player: -----
Place starting influence.
You have selected Austria
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place starting influence.
Remaining influence: 1
Available options:
    3 Canada
    4 UK
    5 Norway
    6 Sweden
    7 Finland
    8 Denmark
    9 Benelux
   10 France
   11 Spain_Portugal
   12 Italy
   13 Greece
   14 Austria
   15 West_Germany
   23 Turkey
> m 7
Selected: Finland
----- US Player: -----
Place starting influence.
You have selected Finland
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place additional starting influence.
Remaining influence: 2
Available options:
    3 Canada
  